(assignment04)=

# Assignment #4 (demo). Exploring OLS, Lasso and Random Forest in a regression task

<img src="https://habrastorage.org/webt/ia/m9/zk/iam9zkyzqebnf_okxipihkgjwnw.jpeg" />

Author: [Yury Kashnitsky](https://www.linkedin.com/in/festline/). All content is distributed under the [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/) license.

**Same assignment as a [Kaggle Notebook](https://www.kaggle.com/kashnitsky/a6-demo-linear-models-and-rf-for-regression) + [solution](https://www.kaggle.com/kashnitsky/a6-demo-regression-solution).**  

```{figure} /_static/img/wine_quality.jpg
:width: 444px
```

**Fill in the missing code and choose answers in [this](https://docs.google.com/forms/d/1aHyK58W6oQmNaqEfvpLTpo6Cb0-ntnvJ18rZcvclkvw/edit) web form.**

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso, LassoCV, LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import (GridSearchCV, cross_val_score,
                                     train_test_split)
from sklearn.preprocessing import StandardScaler

**We are working with UCI Wine quality dataset (no need to download it – it's already there, in course repo and in Kaggle Dataset).**

In [41]:
DATA_PATH = "https://raw.githubusercontent.com/Yorko/mlcourse.ai/master/data/"

In [3]:
data = pd.read_csv(DATA_PATH + "winequality-white.csv", sep=";")

In [7]:
data.tail(10)

fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
4888            6.8             0.220         0.36            1.20      0.052   
4889            4.9             0.235         0.27           11.75      0.030   
4890            6.1             0.340         0.29            2.20      0.036   
4891            5.7             0.210         0.32            0.90      0.038   
4892            6.5             0.230         0.38            1.30      0.032   
4893            6.2             0.210         0.29            1.60      0.039   
4894            6.6             0.320         0.36            8.00      0.047   
4895            6.5             0.240         0.19            1.20      0.041   
4896            5.5             0.290         0.30            1.10      0.022   
4897            6.0             0.210         0.38            0.80      0.020   

      free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
4888                 38.0                 127.0  0.99330  3.04       0.54   
4889                 34.0                 118.0  0.99540  3.07       0.50   
4890                 25.0                 100.0  0.98938  3.06       0.44   
4891                 38.0                 121.0  0.99074  3.24       0.46   
4892                 29.0                 112.0  0.99298  3.29       0.54   
4893                 24.0                  92.0  0.99114  3.27       0.50   
4894                 57.0                 168.0  0.99490  3.15       0.46   
4895                 30.0                 111.0  0.99254  2.99       0.46   
4896                 20.0                 110.0  0.98869  3.34       0.38   
4897                 22.0                  98.0  0.98941  3.26       0.32   

      alcohol  quality  
4888      9.2        5  
4889      9.4        6  
4890     11.8        6  
4891     10.6        6  
4892      9.7        5  
4893     11.2        6  
4894      9.6        5  
4895      9.4        6  
4896     12.8        7  
4897     11.8        6

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


**Separate the target feature, split data in 7:3 proportion (30% form a holdout set, use random_state=17), and preprocess data with `StandardScaler`.**

In [25]:
y = data["quality"]
X = data.drop("quality", axis=1)

X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, train_size=0.7, random_state=17)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)


## Linear regression

**Train a simple linear regression model (Ordinary Least Squares).**

In [26]:
linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train)

LinearRegression()

**<font color='red'>Question 1:</font> What are mean squared errors of model predictions on train and holdout sets?**

In [27]:
MSE_train = mean_squared_error(y_train, linreg.predict(X_train_scaled))
MSE_test = mean_squared_error(y_holdout, linreg.predict(X_holdout_scaled))
print("Mean squared error (train): %.3f" % MSE_train)
print("Mean squared error (test): %.3f" % MSE_test)

Mean squared error (train): 0.558
Mean squared error (test): 0.584


**Sort features by their influence on the target feature (wine quality). Beware that both large positive and large negative coefficients mean large influence on target. It's handy to use `pandas.DataFrame` here.**

**<font color='red'>Question 2:</font> Which feature this linear regression model treats as the most influential on wine quality?**

In [28]:
linreg_coef = pd.DataFrame(
    {"coef": linreg.coef_, "coef_abs": np.abs(linreg.coef_)},
    index=data.columns.drop("quality"),
)
linreg_coef.sort_values(by="coef_abs", ascending=False)

coef  coef_abs
density              -0.665720  0.665720
residual sugar        0.538164  0.538164
volatile acidity     -0.192260  0.192260
pH                    0.150036  0.150036
alcohol               0.129533  0.129533
fixed acidity         0.097822  0.097822
sulphates             0.062053  0.062053
free sulfur dioxide   0.042180  0.042180
total sulfur dioxide  0.014304  0.014304
chlorides             0.008127  0.008127
citric acid          -0.000183  0.000183

## Lasso regression

**Train a LASSO model with $\alpha = 0.01$ (weak regularization) and scaled data. Again, set random_state=17.**

In [31]:
lasso1 = Lasso(alpha=0.01, random_state=17)
lasso1.fit(X_train_scaled, y_train)

Lasso(alpha=0.01, random_state=17)

**Which feature is the least informative in predicting wine quality, according to this LASSO model?**

In [33]:
lasso1_coef = pd.DataFrame(
    {"coef": lasso1.coef_, "coef_abs": np.abs(lasso1.coef_)},
    index=data.columns.drop("quality"),
)
lasso1_coef.sort_values(by="coef_abs", ascending=False)

coef  coef_abs
alcohol               0.322425  0.322425
residual sugar        0.256363  0.256363
density              -0.235492  0.235492
volatile acidity     -0.188479  0.188479
pH                    0.067277  0.067277
free sulfur dioxide   0.043088  0.043088
sulphates             0.029722  0.029722
chlorides            -0.002747  0.002747
fixed acidity        -0.000000  0.000000
citric acid          -0.000000  0.000000
total sulfur dioxide -0.000000  0.000000

**Train LassoCV with random_state=17 to choose the best value of $\alpha$ in 5-fold cross-validation.**

In [34]:
alphas = np.logspace(-6, 2, 200)
lasso_cv = LassoCV(cv=5, alphas=alphas, random_state=17)
lasso_cv.fit(X_train_scaled, y_train)

LassoCV(alphas=array([1.00000000e-06, 1.09698580e-06, 1.20337784e-06, 1.32008840e-06,
       1.44811823e-06, 1.58856513e-06, 1.74263339e-06, 1.91164408e-06,
       2.09704640e-06, 2.30043012e-06, 2.52353917e-06, 2.76828663e-06,
       3.03677112e-06, 3.33129479e-06, 3.65438307e-06, 4.00880633e-06,
       4.39760361e-06, 4.82410870e-06, 5.29197874e-06, 5.80522552e-06,
       6.36824994e-06, 6.98587975e-0...
       1.18953407e+01, 1.30490198e+01, 1.43145894e+01, 1.57029012e+01,
       1.72258597e+01, 1.88965234e+01, 2.07292178e+01, 2.27396575e+01,
       2.49450814e+01, 2.73644000e+01, 3.00183581e+01, 3.29297126e+01,
       3.61234270e+01, 3.96268864e+01, 4.34701316e+01, 4.76861170e+01,
       5.23109931e+01, 5.73844165e+01, 6.29498899e+01, 6.90551352e+01,
       7.57525026e+01, 8.30994195e+01, 9.11588830e+01, 1.00000000e+02]),
        cv=5, random_state=17)

In [35]:
lasso_cv.alpha_ # <= the best alpha

0.0002833096101839324

**<font color='red'>Question 3:</font> Which feature is the least informative in predicting wine quality, according to the tuned LASSO model?**

In [36]:
lasso_cv_coef = pd.DataFrame(
    {"coef": lasso_cv.coef_, "coef_abs": np.abs(lasso_cv.coef_)},
    index=data.columns.drop("quality"),
)
lasso_cv_coef.sort_values(by="coef_abs", ascending=False)

coef  coef_abs
density              -0.648161  0.648161
residual sugar        0.526883  0.526883
volatile acidity     -0.192049  0.192049
pH                    0.146549  0.146549
alcohol               0.137115  0.137115
fixed acidity         0.093295  0.093295
sulphates             0.060939  0.060939
free sulfur dioxide   0.042698  0.042698
total sulfur dioxide  0.012969  0.012969
chlorides             0.006933  0.006933
citric acid          -0.000000  0.000000

**<font color='red'>Question 4:</font> What are mean squared errors of tuned LASSO predictions on train and holdout sets?**

In [37]:
MSE_lasso_train = mean_squared_error(y_train, lasso_cv.predict(X_train_scaled))
MSE_lasso_test = mean_squared_error(y_holdout, lasso_cv.predict(X_holdout_scaled))
print("Mean squared error (train): %.3f" % MSE_lasso_train)
print("Mean squared error (test): %.3f" % MSE_lasso_test)

Mean squared error (train): 0.558
Mean squared error (test): 0.583


## Random Forest

**Train a Random Forest with out-of-the-box parameters, setting only random_state to be 17.**

In [38]:
forest = RandomForestRegressor(random_state=17)
forest.fit(X_train_scaled, y_train)

RandomForestRegressor(random_state=17)

**<font color='red'>Question 5:</font> What are mean squared errors of RF model on the training set, in cross-validation (cross_val_score with scoring='neg_mean_squared_error' and other arguments left with default values) and on holdout set?**

In [40]:
MSE_rf_train = mean_squared_error(y_train, forest.predict(X_train_scaled))
MSE_rf_cv = np.mean(np.abs(cross_val_score(forest, X_train_scaled, y_train, scoring="neg_mean_squared_error")))
MSE_rf_test = mean_squared_error(y_holdout, forest.predict(X_holdout_scaled))
print("Mean squared error (train): %.3f" % MSE_rf_train)
print("Mean squared error (cv): %.3f" % MSE_rf_cv)
print("Mean squared error (test): %.3f" % MSE_rf_test)

Mean squared error (train): 0.053
Mean squared error (cv): 0.414
Mean squared error (test): 0.372


**Tune the `max_features` and `max_depth` hyperparameters with GridSearchCV and again check mean cross-validation MSE and MSE on holdout set.**

In [42]:
forest_params = {'max_depth': list(range(10, 25)),
                 'min_samples_leaf': list(range(1, 8)),
                 'max_features': list(range(6,12))}

locally_best_forest = GridSearchCV(
    RandomForestRegressor(n_jobs=-1, random_state=17),
    forest_params,
    scoring="neg_mean_squared_error",
    n_jobs=-1,
    cv=5,
    verbose=True,
)
locally_best_forest.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 630 candidates, totalling 3150 fits


GridSearchCV(cv=5, estimator=RandomForestRegressor(n_jobs=-1, random_state=17),
             n_jobs=-1,
             param_grid={'max_depth': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
                                       20, 21, 22, 23, 24],
                         'max_features': [6, 7, 8, 9, 10, 11],
                         'min_samples_leaf': [1, 2, 3, 4, 5, 6, 7]},
             scoring='neg_mean_squared_error', verbose=True)

In [43]:
locally_best_forest.best_params_, locally_best_forest.best_score_

({'max_depth': 21, 'max_features': 6, 'min_samples_leaf': 1},
 -0.39773288191505934)

**<font color='red'>Question 6:</font> What are mean squared errors of tuned RF model in cross-validation (cross_val_score with scoring='neg_mean_squared_error' and other arguments left with default values) and on holdout set?**

In [44]:
MSE_tuned_RF_cv = np.mean(np.abs(cross_val_score(locally_best_forest.best_estimator_, X_train_scaled, y_train, scoring="neg_mean_squared_error")))
MSE_tuned_RF_test = mean_squared_error(y_holdout, locally_best_forest.predict(X_holdout_scaled))
print("Mean squared error (cv): %.3f" % MSE_tuned_RF_cv)
print("Mean squared error (test): %.3f" % MSE_tuned_RF_test)

Mean squared error (cv): 0.398
Mean squared error (test): 0.366


**Output RF's feature importance. Again, it's nice to present it as a DataFrame.**<br>
**<font color='red'>Question 7:</font> What is the most important feature, according to the Random Forest model?**

In [47]:
rf_importance = pd.DataFrame(
    locally_best_forest.best_estimator_.feature_importances_,
    columns=["coef"],
    index=data.columns[:-1]
)
rf_importance.sort_values(by="coef", ascending=False) 

coef
alcohol               0.206056
volatile acidity      0.117578
free sulfur dioxide   0.111556
density               0.088549
pH                    0.073659
total sulfur dioxide  0.073640
chlorides             0.073366
residual sugar        0.072072
citric acid           0.062601
fixed acidity         0.061813
sulphates             0.059111

**Make conclusions about the performance of the explored 3 models in this particular prediction task.**